In [ ]:
# Import the required libraries
import re
import httpx
from selectolax.parser import HTMLParser
from dataclasses import dataclass, asdict, fields
from urllib.parse import urljoin, urlsplit
from typing import List, Dict, Optional
import pandas as pd

# Set the site url
site_url: str = "https://scotlandsplaces.gov.uk"

# Set the main url this county that you want to scrape
main_url: str = "https://scotlandsplaces.gov.uk/digital-volumes/ordnance-survey-name-books/lanarkshire-os-name-books-1858-1861"

# Set the csv file path
gazetteer_filepath: str = "./data/GazetteerNames.csv"


# Create a dataclass to store the information
@dataclass
class placename_information:
    page_title: Optional[str] = "No table on page"
    page_number: Optional[int] = 0
    placename: str = "No table on page"
    various_spellings: str = "No table on page"
    authority: str = "No table on page"
    situation: str = "No table on page"
    description: str = "No table on page"
    county: Optional[str] = "No match"


# Create a function to extract the volume number from the url
def extract_volume_number(url: str) -> int:
    return int(url.split("-")[-1].rstrip("/"))


# Create a function to get the volumes
def get_volumes(url: str) -> list[str]:
    try:
        response = httpx.get(url)
        response.raise_for_status()
        print(f"HTTP status: {response.status_code}")
    except httpx.HTTPStatusError as error:
        print(f"HTTP error occurred: {error}")
        return []

    html = HTMLParser(response.text)
    links = html.css("a")

    volume_numbers = []
    for link in links:
        href = link.attributes.get("href")
        if href:
            match = re.search(
                r"/digital-volumes/ordnance-survey-name-books/lanarkshire-os-name-books-1858-1861/lanarkshire-volume-\d+",
                href,
            )
            if match:
                full_url = urljoin(site_url, match.group())
                volume_numbers.append(full_url + "/")

    volume_numbers.sort(key=extract_volume_number)

    return volume_numbers


# Create a function to get the page numbers
def get_page_numbers(volume_url: str) -> list[int]:
    response = httpx.get(volume_url)
    html = HTMLParser(response.text)
    links = html.css("a")

    page_numbers = []
    for link in links:
        href = link.attributes.get("href")
        if href:
            match = re.search(r"\d+$", href)
            if match:
                page_numbers.append(int(match.group()))

    return list(set(page_numbers))


# Create a function to get the html
def get_html(volume_url: str, page: int) -> HTMLParser:
    url = f"{volume_url}{page}"
    response = httpx.get(url)
    return HTMLParser(response.text)


# Create a function to parse the placename information
def parse_placename(html: HTMLParser, page: int) -> list[dict[str, str]]:
    page_header = html.css_first("h1.page-header").text()
    name = html.css("div.well tr")  # get each table row from table
    results = []
    for item in name:
        item_parts = item.css("td")
        if len(item_parts) == 5:  # this will avoid the table headers
            if any(
                re.search(
                    r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
                    part.text().strip(),
                )
                for part in item_parts
            ):
                continue
            new_item = placename_information(
                page_title=page_header,
                page_number=page,
                placename=item_parts[0].text().strip(),
                various_spellings=item_parts[1].text().replace("\n", "|").strip(),
                authority=item_parts[2].text().replace("\n", "|").strip(),
                situation=item_parts[3].text().strip(),
                description=item_parts[4].text().strip(),
                county="LAN",
            )
            results.append(asdict(new_item))
    # print(results)
    return results

# Get the county from the url
def get_county_from_url(url: str) -> str:
    split_url = urlsplit(url)
    path_parts = split_url.path.split('/')
    county = path_parts[3].split('-')[0].upper()
    return county

# Load csv file into a pandas dataframe and return the dataframe where local_authority is like the county
def load_csv(filepath: str, county: str) -> pd.DataFrame:
    df = pd.read_csv(filepath, low_memory=False)
    df['local_authority'] = df['local_authority'].str.upper()
    df = df[df['local_authority'].str.contains(county)]
    return df   


# Convert the list of dictionaries to a pandas dataframe
def to_dataframe(res: List[Dict[str, str]]) -> pd.DataFrame:
    fieldnames = [field.name for field in fields(placename_information)]
    df = pd.DataFrame(res, columns=fieldnames)
    return df


# Output data to spreadsheet
def write_to_excel(data: pd.DataFrame, output_filepath: str) -> None:
    with pd.ExcelWriter(
        output_filepath,
        engine="xlsxwriter",
        datetime_format="dd mmm yyyy hh:mm:ss",
        date_format="dd mmm yyyy",
    ) as writer:
        data.to_excel(
            writer,
            sheet_name="Sheet1",
            index=False,
            header=False,
            startrow=1,
        )
        workbook = writer.book
        worksheet = writer.sheets["Sheet1"]
        header_format = workbook.add_format(
            {
                "bold": True,
                "text_wrap": True,
                "valign": "top",
                "fg_color": "#9FC5E8",
                "border": 1,
            }
        )
        for col_num, value in enumerate(data.columns):
            worksheet.write(0, col_num, value, header_format)
        print("Spreadsheet created.")


# Create a main function
def main() -> None:
    county = get_county_from_url(main_url)
    df = load_csv(gazetteer_filepath, county)

    volumes = get_volumes(main_url)
    print(volumes)
    for volume in volumes:
        volume_name = urlsplit(volume).path.split("/")[-2]
        page_numbers = get_page_numbers(volume)
        print(page_numbers)
        results = []  # Define results here
        for page in page_numbers:
            html = get_html(volume, page)
            page_results = parse_placename(html, page)  # Pass page to parse_placename
            print(f"Results for volume {volume_name}, page {page}: {page_results}")
            results.extend(page_results)  # Add page_results to results
        df = to_dataframe(results)  # Pass results to to_dataframe
        write_to_excel(df, f"./output/{volume_name}.xlsx")


if __name__ == "__main__":
    main()